# Data Modeling with Cassandra

## ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [47]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [48]:
# Current working directory
print(os.getcwd())

# Current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# A for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


**Processing files to create the data file csv that will be used for Apache Casssandra tables**

In [49]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [50]:
# Number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [51]:
# Connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish connection 
session = cluster.connect()

#### Create Keyspace

In [52]:
# Create a Keyspace 
try:

    session.execute(""" 
    CREATE KEYSPACE IF NOT EXISTS farouk
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
        print(e)

#### Set Keyspace

In [53]:
# Set KEYSPACE 
try:

    session.set_keyspace('farouk') 
except Exception as e:
    print(e)


## Creating , Inserting data and running queries on three Distinct tables

**1a. Create Music_app table**

In [54]:
#CREATING TABLE FOR MUSIC_APP
query = "CREATE TABLE IF NOT EXISTS Music_app"
query = query + """(
    sessionId int,
    itemInSession int,
    artist text, 
    firstName text, 
    lastName text, 
    length float,
    song text, 
    PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query) 
except Exception as e:
    print(e)                

**1b. Read and Insert data from csv file to the Music_app table**

In [55]:
# CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#INSERT statements 
        query = "INSERT INTO Music_app (sessionId,itemInSession,artist, firstName,lastName,length,song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query,(int(line[8]),int(line[3]),line[0],line[1],line[4],float(line[5]),line[9]))

**1c. Select and Display artist, song and song's length with a given session Id**

In [56]:
#SELECT statement 
query ="SELECT artist, song, length  FROM music_app WHERE sessionid = 338"
try:
        rows = session.execute(query)
except Exception as e:
        print(e)
for row in rows:
        print ("ARTIST:",row.artist, "SONG:",row.song, "LENGTH:",row.length)

ARTIST: Pixies SONG: Build High LENGTH: 89.36444091796875
ARTIST: The Roots / Jack Davey SONG: Atonement LENGTH: 155.95057678222656
ARTIST: Mike And The Mechanics SONG: A Beggar On A Beach Of Gold LENGTH: 275.1211853027344
ARTIST: Faithless SONG: Music Matters (Mark Knight Dub) LENGTH: 495.30731201171875


**2a. Create  Artist_song table**

In [63]:
#CREATING TABLE FOR ARTIST_SONG
query = "CREATE TABLE IF NOT EXISTS Artist_song"
query = query + """(
    userId int,
    sessionId int,
    itemInSession int,
    artist text, 
    firstName text, 
    lastName text, 
    song text,
    PRIMARY KEY ((userid, sessionid), iteminsession))"""
try:
        session.execute(query) 
except Exception as e:
        print(e)

**2b. Read and Insert data from csv file to the Artist_song table**

In [64]:
# CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#INSERT statements 
        query = "INSERT INTO Artist_song (userid,sessionid,iteminsession,artist,firstname,lastname,song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0],line[1],line[4],line[9]))

**2c. Select and Display artist,songtitle and user name (first and last) with a given user Id** 

In [70]:
#SELECT statement 
query = "SELECT artist, firstname, lastname,song FROM artist_song WHERE userid = 10 AND sessionid =182"
try:
        rows = session.execute(query)
except Exception as e:
        print(e)
for row in rows:
        print ("ARTIST:",row.artist, "FISRT NAME:",row.firstname, "LAST NAME:",row.lastname,"SONG:",row.song)

ARTIST: Down To The Bone FISRT NAME: Sylvie LAST NAME: Cruz SONG: Keep On Keepin' On
ARTIST: Three Drives FISRT NAME: Sylvie LAST NAME: Cruz SONG: Greece 2000
ARTIST: Sebastien Tellier FISRT NAME: Sylvie LAST NAME: Cruz SONG: Kilometer
ARTIST: Lonnie Gordon FISRT NAME: Sylvie LAST NAME: Cruz SONG: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


**3a. Create User_names table**

In [60]:
#CREATING TABLE FOR USER_NAMES
query = "CREATE TABLE IF NOT EXISTS User_names"
query = query + """(
    song text, 
    userId int,
    firstName text,
    lastName text, 
    PRIMARY KEY (song, userId))"""
try:
        session.execute(query) 
except Exception as e:
        print(e)

**3b. Read and Insert data from csv file to the User_names table**

In [61]:
# CSV file
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#INSERT statements 
        query = "INSERT INTO user_names (song,userid,firstname,lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query,(line[9],int(line[10]),line[1],line[4]))

**3c. Select and Display every user name (first and last) who listened to a specific song** 

In [62]:
#SELECT statement 
query = "SELECT firstname, lastname FROM user_names WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("FIRST NAME:",row.firstname, "LAST NAME:", row.lastname)

FIRST NAME: Jacqueline LAST NAME: Lynch
FIRST NAME: Tegan LAST NAME: Levine
FIRST NAME: Sara LAST NAME: Johnson


**Dropping tables**

In [26]:
#Dropping the tables 
query = "drop table Music_app"
query = "drop table Artist_song"
query = "drop table User_names"
try:
    rows = session.execute(query) 
except Exception as e:
    print(e)

**Closing the session and cluster connection**

In [27]:
session.shutdown()
cluster.shutdown()